In [1]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.1 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-orc8056q
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-orc8056q
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369501 sha256=65f8c0336a2156f2a587e682b47ee0b06ec6bab8f67498cd642f19fa5ecae6b8
  Stored in directory: /tmp/pip-ephem-wheel-cache-8sntu89l/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [2]:
import torch
import clip
from PIL import Image
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from torch.nn.utils.rnn import pad_sequence
from torch.nn import functional as F
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from tensorflow import keras
import tensorflow as tf

from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive


In [3]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
model, CliPPreprocess = clip.load('ViT-B/32')

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 126MiB/s]


In [6]:
csv_path = '/content/drive/MyDrive/results.csv'
dataset = pd.read_csv(csv_path, delimiter = '|')
dataset.columns = ['image_name', 'comment_number', 'comment']
dataset['comment_number'] = dataset['comment_number'].str.strip()
dataset = dataset.drop([19995,19996,19997,19998,19999])
dataset['comment_number'] = pd.to_numeric(dataset['comment_number'])
dataset['comment'] = dataset['comment'].str.strip()

captions = dataset['comment'].tolist()

words = []
for caption in captions:
  words.extend(caption.split())
words = [word.lower() for word in words]

from collections import Counter
word_counts = Counter(words)

vocab = []
for word,freq in word_counts.items():
  if freq >= 3:
    vocab.append(word)
vocab = ['<PAD>', '<UNK>','<START>','<END>'] + vocab

word_to_index = {word:index for index,word in enumerate(vocab)}
index_to_word = {index:word for index,word in enumerate(vocab)}

def caption_to_indices(captions, word_to_index):
    # iterate over each caption
    captions_numerical = []
    for caption in captions:
        caption_indices = []
        caption_indices.append(word_to_index['<START>'])
        caption_indices.extend([word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in caption.lower().split()])
        caption_indices.append(word_to_index['<END>'])
        captions_numerical.append(caption_indices)
    return captions_numerical

In [ ]:
import numpy as np

CLIPOutputs_path = '/content/drive/MyDrive/image_representations.npy'
image_representations = np.load(CLIPOutputs_path, allow_pickle = True)

padded_captions_path = '/content/drive/MyDrive/Flickr30kCaptions.npy'
padded_captions = np.load(padded_captions_path, allow_pickle = True)

tensors = [t.squeeze() for t in image_representations]
image_representations = torch.stack(tensors)

In [ ]:
padded_captions = tf.reshape(padded_captions, [31782,5,84])

In [ ]:
import zipfile
import io
import numpy as np
from PIL import Image

zip_path = '/content/drive/MyDrive/flickr30k_images.zip'

# Open the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    image_files = [name for name in zip_ref.namelist() if name.endswith('.jpg')]

    # Create an empty NumPy array to store the images
    images = np.empty(len(image_files), dtype=object)

    # Loop over the image file names
    for i, file_name in enumerate(image_files):
        # Read the image data from the zip file
        with zip_ref.open(file_name) as image_file:
            # Open the image using PIL
            image = Image.open(io.BytesIO(image_file.read()))

            # Store the processed image in the array
            images[i] = image


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: ignored

In [ ]:
# Remove image 4000
images = np.delete(images, 3999, axis=0)

In [ ]:
representations = []
i=0
# Process each image and generate representations
for image in images:
    # Preprocess the image using the preprocess function from clip.load
    preprocessed_image = CliPPreprocess(image).unsqueeze(0).to('cuda')
    # Generate the image representation
    with torch.no_grad():
        representation = model.encode_image(preprocessed_image).float()

    # Append the representation to the list
    representations.append(representation)

# Convert the list of representations to a NumPy array
representations_np = np.array(representations)

# Define the path to save the representations
representations_path = '/content/drive/MyDrive/CLIP_outputs.npy'  # Modify as desired

# Save the representations as a .npy file
np.save(representations_path, representations_np)

Streaming output truncated to the last 5000 lines.
26783
26784
26785
26786
26787
26788
26789
26790
26791
26792
26793
26794
26795
26796
26797
26798
26799
26800
26801
26802
26803
26804
26805
26806
26807
26808
26809
26810
26811
26812
26813
26814
26815
26816
26817
26818
26819
26820
26821
26822
26823
26824
26825
26826
26827
26828
26829
26830
26831
26832
26833
26834
26835
26836
26837
26838
26839
26840
26841
26842
26843
26844
26845
26846
26847
26848
26849
26850
26851
26852
26853
26854
26855
26856
26857
26858
26859
26860
26861
26862
26863
26864
26865
26866
26867
26868
26869
26870
26871
26872
26873
26874
26875
26876
26877
26878
26879
26880
26881
26882
26883
26884
26885
26886
26887
26888
26889
26890
26891
26892
26893
26894
26895
26896
26897
26898
26899
26900
26901
26902
26903
26904
26905
26906
26907
26908
26909
26910
26911
26912
26913
26914
26915
26916
26917
26918
26919
26920
26921
26922
26923
26924
26925
26926
26927
26928
26929
26930
26931
26932
26933
26934
26935
26936
26937
26938
26939
26940
2

<ipython-input-74-ffcd4e824075>:22: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  representations_np = np.array(representations)
<ipython-input-74-ffcd4e824075>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  representations_np = np.array(representations)


In [ ]:
numerical_captions = caption_to_indices(captions,word_to_index)

numerical_captions_tensors = [torch.tensor(caption) for caption in numerical_captions]
padded_captions = pad_sequence(numerical_captions_tensors, batch_first = True)

np.save('/content/drive/MyDrive/Flickr30kCaptions.npy', padded_captions)

#important variables are now padded_captions (tensor) and representations_np (numpy array)